In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
# from sklearn.metrics import roc_auc_score
# from fastai.vision.all import *
import wandb
# import timm

In [ ]:
# approach = ''

In [2]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchinfo import summary
# # from fastai.callback.wandb import *
# from torch.cuda.amp import autocast, GradScaler

In [3]:
# from fastai.vision.all import *
from fastai.tabular.all import *
# from fastai.callback.wandb import WandbCallback

In [2]:
# import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
set_seed(42, reproducible=True)

In [5]:
datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [6]:
df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)

In [7]:
df.isnull().sum().any()

False

In [8]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250000 entries, 0 to 249999
Data columns (total 101 columns):
 #    Column  Non-Null Count   Dtype  
---   ------  --------------   -----  
 0    f0      250000 non-null  float64
 1    f1      250000 non-null  int64  
 2    f2      250000 non-null  float64
 3    f3      250000 non-null  float64
 4    f4      250000 non-null  float64
 5    f5      250000 non-null  float64
 6    f6      250000 non-null  float64
 7    f7      250000 non-null  float64
 8    f8      250000 non-null  float64
 9    f9      250000 non-null  float64
 10   f10     250000 non-null  float64
 11   f11     250000 non-null  float64
 12   f12     250000 non-null  float64
 13   f13     250000 non-null  float64
 14   f14     250000 non-null  float64
 15   f15     250000 non-null  float64
 16   f16     250000 non-null  int64  
 17   f17     250000 non-null  float64
 18   f18     250000 non-null  float64
 19   f19     250000 non-null  float64
 20   f20     250000 non-null 

<ipython-input-8-6ac79f7ef903>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


So no NaNs.

In [36]:
# df.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,0.511213,51.378476,0.107155,0.050010,3595.133426,8.205953,164.508753,0.375533,16.669745,1.190382,...,4856.812768,22.579100,2.030554,0.079692,1.555097,2.417556,0.537484,1.576900,8.048805,6.813920
std,0.307884,42.396636,1.322200,0.792368,6072.401061,5.475723,183.335563,0.813597,99.758709,0.099700,...,8501.609009,14.849390,0.900211,0.587780,9.253785,0.892563,0.226589,0.646306,5.647368,7.940179
min,-0.069273,-17.000000,-7.895580,-1.475560,-7589.280000,-3.291050,-40.967200,-4.143080,-502.813000,0.934037,...,-12695.700000,-4.059170,0.057800,-1.998800,-24.686300,-1.131980,0.005249,-0.646967,-0.842397,0.000000
25%,0.251287,18.000000,-0.611172,-0.719418,163.864750,4.110127,27.894900,-0.026245,-17.392025,1.132640,...,73.203100,11.525450,1.471650,-0.408975,-4.004925,1.906718,0.359646,1.215810,3.732800,1.000000
50%,0.514962,41.000000,0.253815,0.004099,943.000500,7.472445,91.005250,0.619862,8.714945,1.170370,...,1060.025000,19.993200,1.660830,0.215710,0.759942,2.340430,0.531348,1.451285,7.182205,4.000000
75%,0.777322,75.000000,0.759249,0.765456,4115.355000,11.030950,240.843750,0.933855,55.407625,1.218880,...,5572.982500,32.271625,2.320085,0.503134,6.202503,2.910020,0.709807,1.901632,10.998550,10.000000
max,1.072070,273.000000,9.768590,1.680190,37847.500000,35.078000,947.143000,4.010380,465.956000,1.712450,...,54334.600000,79.912400,5.403020,1.944190,42.890400,5.576040,1.105400,4.492620,34.019200,42.000000


In [9]:
y = df.loss

In [10]:
features = [x for x in df.columns if x != 'loss']

In [11]:
X = df[features]

In [12]:
len(X)

250000

In [13]:
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [14]:
len(y), len(y.value_counts())

(250000, 43)

So, it seems that we're actually looking at a classification problem here -- not a regression one.

# Decision Tree (baseline)

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

Now, let it overfit:

In [16]:
model = DecisionTreeClassifier(random_state=42)

In [17]:
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [18]:
y_pred = model.predict(X_valid)

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
cross_val_score(model, X, y, cv=5)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([0.09832, 0.09872, 0.09842, 0.09776, 0.09964])

In [26]:
training = X_train.join(y_train)
training.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
75381,0.660780,11,1.025300,-0.899833,135.097,26.59640,-3.57996,1.115800,28.70010,1.11695,...,831.0820,6.12317,1.54644,-0.847670,-6.23057,1.89005,0.334040,1.53859,29.42350,10
65569,0.632091,123,-0.740867,0.969644,405.789,3.45397,162.87600,-0.008684,-3.55616,1.21623,...,13801.7000,16.05840,4.20007,1.327930,-1.09445,2.08871,0.221534,1.41467,2.37455,4
163473,0.728140,16,6.831980,0.194010,7190.240,2.21486,22.96890,2.050900,-43.12080,1.55845,...,770.4300,9.87490,1.10869,-0.665988,11.21250,1.91561,0.566240,1.51815,8.72715,3
90518,0.549641,125,0.825102,-0.403810,-317.322,4.94806,180.60300,0.563666,-14.81110,1.17739,...,715.5700,8.32496,3.94989,-0.658169,-3.44205,1.61060,0.778453,1.81879,10.21320,7
138866,0.097337,72,0.891514,0.974637,2514.030,12.08510,530.23800,-0.937181,-6.19168,1.54768,...,-75.2506,13.24720,1.74937,0.509452,-1.65423,2.80576,0.565295,2.10123,12.99470,0


In [28]:
y_train_pred = model.predict(X_train)
y_train_pred[:5]

array([10,  4,  3,  7,  0])

So indeed, some crazy overfitting happening. Let's try a more plausible set of hyperparameters.

In [29]:
model = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)

In [30]:
model.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_leaf=2, random_state=42)

In [31]:
y_pred = model.predict(X_valid)

In [32]:
cross_val_score(model, X, y, cv=5)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([0.11166, 0.11182, 0.11214, 0.11112, 0.1112 ])

In [33]:
y_valid[:15], y_pred[:15]

(id
 38683      0
 64939      7
 3954      34
 120374     1
 172861     6
 149303    22
 111626     5
 164553     3
 55779      0
 141990     0
 176728     4
 22049      0
 77431      7
 111499    20
 228764     0
 Name: loss, dtype: int64,
 array([ 2,  2,  1,  6,  0,  2,  0,  3, 10,  0,  0,  5, 12,  1, 10]))

That performs rather worse.

In [34]:
%config Completer.use_jedi = False

In [36]:
model.get_depth()

62

Let's try running it one more time, now with a specified max depth of 20.

In [37]:
model = DecisionTreeClassifier(max_depth=20, random_state=42)

In [38]:
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=20, random_state=42)

In [39]:
y_pred = model.predict(X_valid)

In [40]:
cross_val_score(model, X, y, cv=5)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([0.2112 , 0.2125 , 0.20882, 0.21512, 0.21638])

In [41]:
model = RandomForestClassifier(random_state=42, n_estimators=100, min_samples_leaf=2)

In [42]:
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=2, random_state=42)

In [43]:
y_pred = model.predict(X_valid)

In [44]:
cross_val_score(model, X, y, cv=5)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


array([0.2441 , 0.24364, 0.24354, 0.24366, 0.24368])

In [46]:
model.score?

Signature: model.score(X, y, sample_weight=None)
Docstring:
Return the mean accuracy on the given test data and labels.

In multi-label classification, this is the subset accuracy
which is a harsh metric since you require for each sample that
each label set be correctly predicted.

Parameters
----------
X : array-like of shape (n_samples, n_features)
    Test samples.

y : array-like of shape (n_samples,) or (n_samples, n_outputs)
    True labels for `X`.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

Returns
-------
score : float
    Mean accuracy of ``self.predict(X)`` wrt. `y`.
File:      ~/anaconda3/envs/tabular/lib/python3.8/site-packages/sklearn/base.py
Type:      method


So, the default score for the `RandomForestClassifier` is accuracy -- meaning that the `RandomForestClassifier` with these hyperparameters gets about 25% accuracy. Not exactly great (but better than the horrifically overfit `DecisionTreeClassifier`, and about 4% better than the `DecisionTreeClassifier` with saner hyperparameters.

Note that the metric being used for the task is RMSE -- so maybe consider a regressor?

In [48]:
from sklearn.ensemble import RandomForestRegressor

In [49]:
?RandomForestRegressor

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the fore

In [50]:
model = RandomForestRegressor(random_state=42, n_estimators=100)

In [51]:
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [52]:
y_pred = model.predict(X_valid)

In [54]:
scores = cross_val_score(model, X, y, cv=5)

In [55]:
scores

array([-0.01167757, -0.01353459, -0.01139984, -0.01034689, -0.00996452])

In [57]:
rmses = [math.sqrt(abs(score)) for score in scores]

In [58]:
rmses

[0.10806282456243882,
 0.11633824882282971,
 0.10677005441820361,
 0.10171965394414972,
 0.09982244295873825]

Let's give that a submit...

In [59]:
test_df = pd.read_csv(datapath/'test.csv', index_col='id', low_memory=False)

In [60]:
test_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [62]:
X_test = test_df[features] # this is just for naming consistency

In [68]:
y_test_preds = model.predict(X_test)

In [70]:
y_test_preds[:15]

array([ 6.86,  8.42,  6.61,  5.18,  8.86, 11.54,  8.29,  7.55,  5.35,
        8.58,  7.89,  7.71,  3.07,  4.82, 10.54])

In [72]:
test_df['loss'] = y_test_preds

In [74]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [78]:
sample_df.loc[:, 'loss'] = y_test_preds

In [79]:
sample_df.head()

,id,loss
0,250000,6.86
1,250001,8.42
2,250002,6.61
3,250003,5.18
4,250004,8.86


In [80]:
sample_df.to_csv('20210806_RandomForest_100estimators.csv', index=False)